# Utilities

In the general introduction we covered the basics of the TOAST data and processing model.  In this notebook we cover several sets of utilities that are included within TOAST that can be used when constructing new Operators or working with the data interactively.  Often these utilities make use of compiled code "under the hood" for performance.  For example:

- `toast.rng`:  Streamed random number generation, with support for generating random samples from any location within a stream.

- `toast.qarray`:  Vectorized quaternion operations.

- `toast.fft`:  API Wrapper around different vendor FFT packages.

- `toast.healpix`:  Subset of pixel projection routines, simd vectorized and threaded.

- `toast.timing`:  Simple serial timers, global named timers per process, a decorator to time calls to functions, and MPI tools to gather timing statistics from multiple processes.

First we import some packages we will use in this notebook.

In [ ]:
# Built-in modules
import sys
import os

# External modules
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u

# TOAST
import toast


# Capture C++ output in the jupyter cells
%load_ext wurlitzer

# Display inline plots
%matplotlib inline

### Random Number Example

Here is a quick example of a threaded generation of random numbers drawn from a unit-variance gaussian distribution.  Note the "key" pair of uint64 values and the first value of the "counter" pair determine the stream, and the second value of the counter pair is effectively the sample in that stream.  We can drawn randoms from anywhere in the stream in a reproducible fashion (i.e. this random generator is stateless).  Under the hood, this uses the Random123 package on each thread.

In [ ]:
import toast.rng as rng

# Number of random samples
nrng = 10

In [ ]:
# Draw randoms from the beginning of a stream
rng1 = rng.random(
    nrng, key=[12, 34], counter=[56, 0], sampler="gaussian", threads=True
)

In [ ]:
# Draw randoms from some later starting point in the stream
rng2 = rng.random(
    nrng, key=[12, 34], counter=[56, 4], sampler="gaussian", threads=True
)

In [ ]:
# The returned objects are buffer providers, so can be used like a numpy array.
print("Returned RNG buffers:")
print(rng1)
print(rng2)

In [ ]:
# Compare the elements.  Note how the overlapping sample indices match.  The
# randoms drawn for any given sample agree regardless of the starting sample.
print("------ rng1 ------")
for i in range(nrng):
    print("rng1 {}:  {}".format(i, rng1[i]))
print("------ rng2 ------")
for i in range(nrng):
    print("rng2 {}:  {}".format(i + 4, rng2[i]))

### Quaternion Array Example

The quaternion manipulation functions internally attempt to improve performance using OpenMP SIMD directives and threading in cases where it makes sense.  The Python API is modelled after the quaternionarray package (https://github.com/zonca/quaternionarray/).  There are functions for common operations like multiplying quaternion arrays, rotating arrays of vectors, converting to and from angle representations, SLERP, etc.

In [ ]:
import toast.qarray as qa

# Number points for this example

nqa = 5

In [ ]:
# Make some fake rotation data by sweeping through theta / phi / pa angles

theta = np.linspace(0.0, np.pi, num=nqa)
phi = np.linspace(0.0, 2 * np.pi, num=nqa)
pa = np.zeros(nqa)
print("----- input angles -----")
print("theta = ", theta)
print("phi = ", phi)
print("pa = ", pa)

In [ ]:
# Convert to quaternions

quat = qa.from_angles(theta, phi, pa)

print("\n----- output quaternions -----")
print(quat)

In [ ]:
# Use these to rotate a vector

zaxis = np.array([0.0, 0.0, 1.0])
zrot = qa.rotate(quat, zaxis)

print("\n---- Z-axis rotated by quaternions ----")
print(zrot)

In [ ]:
# Rotate different vector by each quaternion

zout = qa.rotate(quat, zrot)

print("\n---- Arbitrary vectors rotated by quaternions ----")
print(zout)

In [ ]:
# Multiply two quaternion arrays

qcopy = np.array(quat)
qout = qa.mult(quat, qcopy)

print("\n---- Product of two quaternion arrays ----")
print(qout)

In [ ]:
# SLERP quaternions

qtime = 3.0 * np.arange(nqa)
qtargettime = np.arange(3.0 * (nqa - 1) + 1)
qslerped = qa.slerp(qtargettime, qtime, quat)

print("\n---- SLERP input ----")
for t, q in zip(qtime, quat):
    print("t = {} : {}".format(t, q))
    
print("\n---- SLERP output ----")
for t, q in zip(qtargettime, qslerped):
    print("t = {} : {}".format(t, q))

### FFT Example

The internal FFT functions in TOAST are very limited and focus only on batched 1D Real FFTs.  These are used for simulated noise generation and timestream filtering.  Internally the compiled code can use either FFTW or MKL for the backend calculation.

In [ ]:
# Number of batched FFTs

nbatch = 5

# FFT length

nfft = 65536

In [ ]:
# Create some fake data

infft = np.zeros((nbatch, nfft), dtype=np.float64)
for b in range(nbatch):
    infft[b, :] = rng.random(nfft, key=[0, 0], counter=[b, 0], sampler="gaussian")

print("----- FFT input -----")
print(infft)

In [ ]:
# Forward FFT

outfft = toast.fft.r1d_forward(infft)

print("\n----- FFT output -----")
print(outfft)

In [ ]:
# Reverse FFT

backfft = toast.fft.r1d_backward(outfft)

print("\n----- FFT inverse output -----")
print(backfft)